In [1]:
import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
#from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', 500)

In [2]:
pos = pd.read_csv('POS_CASH_balance.csv')

In [3]:
pos.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
pos['NAME_CONTRACT_STATUS'] = le.fit_transform(pos['NAME_CONTRACT_STATUS'].astype(str))

In [8]:
pos['NAME_CONTRACT_STATUS'].unique()

array([0, 4, 7, 2, 6, 5, 3, 8, 1])

In [6]:
e =pos.groupby('SK_ID_CURR').SK_DPD.sum().reset_index()
r =pos.groupby('SK_ID_CURR').SK_DPD_DEF.sum().reset_index()
t =pos.groupby('SK_ID_CURR').SK_ID_PREV.agg(lambda SK_ID_PREV: len(SK_ID_PREV.unique())).reset_index()
q = pos[['SK_ID_CURR', 'NAME_CONTRACT_STATUS']].groupby('SK_ID_PREV').nunique().reset_index()
w = pos[['SK_ID_CURR', 'NAME_CONTRACT_STATUS']].groupby('SK_ID_PREV').max().reset_index()

KeyError: 'SK_ID_PREV'

In [ ]:
# Number of Loans per Customer
NumLoanPerCust = bureau[['SK_ID_CURR', 'DAYS_CREDIT']].groupby(by = ['SK_ID_CURR'])['DAYS_CREDIT'].count().reset_index().rename(index=str, columns={'DAYS_CREDIT': 'BUREAU_LOAN_COUNT'})
# Number of types of Credit loans for each Customer 
NumTypeofLoansPerCust = bureau[['SK_ID_CURR', 'CREDIT_TYPE']].groupby(by = ['SK_ID_CURR'])['CREDIT_TYPE'].nunique().reset_index().rename(index=str, columns={'CREDIT_TYPE': 'BUREAU_LOAN_TYPES'})

bureau = bureau.merge(NumLoanPerCust, on = ['SK_ID_CURR'], how = 'left')
bureau = bureau.merge(NumTypeofLoansPerCust, on = ['SK_ID_CURR'], how = 'left')

# Average Number of Loans per Loan Type
bureau['AVERAGE_LOAN_TYPE'] = bureau['BUREAU_LOAN_COUNT']/bureau['BUREAU_LOAN_TYPES']
bureau['CREDIT_ACTIVE_BINARY'] = bureau['CREDIT_ACTIVE']
bureau['CREDIT_ACTIVE_BINARY'] = bureau.apply(lambda x: f(x.CREDIT_ACTIVE), axis = 1)

# Calculate mean number of loans that are ACTIVE per CUSTOMER 
MeanNumofLoanActiveCust = bureau.groupby(by = ['SK_ID_CURR'])['CREDIT_ACTIVE_BINARY'].mean().reset_index().rename(index=str, columns={'CREDIT_ACTIVE_BINARY': 'ACTIVE_LOANS_PERCENTAGE'})

# Groupby each Customer and Sort values of DAYS_CREDIT in ascending order
DaysCredit = bureau[['SK_ID_CURR', 'SK_ID_BUREAU', 'DAYS_CREDIT']].groupby(by = ['SK_ID_CURR'])
DaysCredit2 = DaysCredit.apply(lambda x: x.sort_values(['DAYS_CREDIT'], ascending = False)).reset_index(drop = True)


# Calculate Difference between the number of Days 
DaysCredit2['DAYS_CREDIT1'] = DaysCredit2['DAYS_CREDIT']*-1
DaysCredit2['DAYS_DIFF'] = DaysCredit2.groupby(by = ['SK_ID_CURR'])['DAYS_CREDIT1'].diff()
DaysCredit2['DAYS_DIFF'] = DaysCredit2['DAYS_DIFF'].fillna(0).astype('uint32')

bureau['CREDIT_ENDDATE_BINARY'] = bureau['DAYS_CREDIT_ENDDATE']
bureau['CREDIT_ENDDATE_BINARY'] = bureau.apply(lambda x: k(x.DAYS_CREDIT_ENDDATE), axis = 1)

#% of LOANS PER CUSTOMER WHERE END DATE FOR CREDIT IS PAST
LoansEndDate = bureau.groupby(by = ['SK_ID_CURR'])['CREDIT_ENDDATE_BINARY'].mean().reset_index().rename(index=str, columns={'CREDIT_ENDDATE_BINARY': 'CREDIT_ENDDATE_PERCENTAGE'})


bureau['AMT_CREDIT_SUM_DEBT'] = bureau['AMT_CREDIT_SUM_DEBT'].fillna(0)
bureau['AMT_CREDIT_SUM'] = bureau['AMT_CREDIT_SUM'].fillna(0)

SumCredDebt = bureau[['SK_ID_CURR', 'AMT_CREDIT_SUM_DEBT']].groupby(by = ['SK_ID_CURR'])['AMT_CREDIT_SUM_DEBT'].sum().reset_index().rename( index = str, columns = { 'AMT_CREDIT_SUM_DEBT': 'TOTAL_CUSTOMER_DEBT'})
SumCredDebt2 = bureau[['SK_ID_CURR', 'AMT_CREDIT_SUM']].groupby(by = ['SK_ID_CURR'])['AMT_CREDIT_SUM'].sum().reset_index().rename( index = str, columns = { 'AMT_CREDIT_SUM': 'TOTAL_CUSTOMER_CREDIT'})

bureau = bureau.merge(SumCredDebt, on = ['SK_ID_CURR'], how = 'left')
bureau = bureau.merge(SumCredDebt2, on = ['SK_ID_CURR'], how = 'left')

bureau['DEBT_CREDIT_RATIO'] = bureau['TOTAL_CUSTOMER_DEBT']/bureau['TOTAL_CUSTOMER_CREDIT']
bureau['AMT_CREDIT_SUM_DEBT'] = bureau['AMT_CREDIT_SUM_DEBT'].fillna(0)
bureau['AMT_CREDIT_SUM_OVERDUE'] = bureau['AMT_CREDIT_SUM_OVERDUE'].fillna(0)

AMT_CREDIT_SUM_DEBT = bureau[['SK_ID_CURR', 'AMT_CREDIT_SUM_DEBT']].groupby(by = ['SK_ID_CURR'])['AMT_CREDIT_SUM_DEBT'].sum().reset_index().rename( index = str, columns = { 'AMT_CREDIT_SUM_DEBT': 'TOTAL_CUSTOMER_DEBT'})
AMT_CREDIT_SUM_OVERDUE = bureau[['SK_ID_CURR', 'AMT_CREDIT_SUM_OVERDUE']].groupby(by = ['SK_ID_CURR'])['AMT_CREDIT_SUM_OVERDUE'].sum().reset_index().rename( index = str, columns = { 'AMT_CREDIT_SUM_OVERDUE': 'TOTAL_CUSTOMER_OVERDUE'})

bureau = bureau.merge(AMT_CREDIT_SUM_DEBT, on = ['SK_ID_CURR'], how = 'left')
bureau = bureau.merge(AMT_CREDIT_SUM_OVERDUE, on = ['SK_ID_CURR'], how = 'left')
bureau['OVERDUE_DEBT_RATIO'] = bureau['TOTAL_CUSTOMER_OVERDUE_y']/bureau['TOTAL_CUSTOMER_DEBT']

CNT_CREDIT_PROLONG = bureau[['SK_ID_CURR', 'CNT_CREDIT_PROLONG']].groupby(by = ['SK_ID_CURR'])['CNT_CREDIT_PROLONG'].mean().reset_index().rename( index = str, columns = { 'CNT_CREDIT_PROLONG': 'AVG_CREDITDAYS_PROLONGED'})
bureau = bureau.merge(CNT_CREDIT_PROLONG, on = ['SK_ID_CURR'], how = 'left')